<a href="https://colab.research.google.com/github/nguyen-nhat-mai/methane-leak-detection/blob/main/Web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build web app for classifier model

In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00


In [2]:
!pip install streamlit_folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.4/423.4 kB 26.0 MB/s eta 0:00:00


Reference:

https://towardsdatascience.com/create-an-image-classification-web-app-using-pytorch-and-streamlit-f043ddf00c24#2b4c
https://www.youtube.com/watch?v=NEhrkeF2o_M

In [49]:
# Write the web app
%%writefile app.py
import streamlit as st
from PIL import Image
from torchvision import models, transforms
import torch
import time
import pandas as pd
import base64
from io import BytesIO
import altair as alt
import folium
from streamlit_folium import st_folium, folium_static

#---------------------- TO UPDATE WITH FINAL MODEL-----------------------#

def predict(image_path):
    # Load model
    best_model = models.resnet101(pretrained=True)
    # Define transformation
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
      )])
    # Load data
    img = Image.open(image_path)
    batch_t = torch.unsqueeze(transform(img), 0)
    # Do inference
    best_model.eval()
    out = best_model(batch_t)
    # Load all of the classes => Yes/ No for methane case
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]
    # Calculate the probability
    prob = torch.nn.functional.softmax(out, dim=1)[0] * 100
    _, indices = torch.sort(out, descending=True)
    # Return the classes and corresponding probability
    return [(classes[idx], prob[idx].item()) for idx in indices[0][:1]]

# ------------------------------WEB APP----------------------------------#

# Define a function to create a download link for a given DataFrame
def download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode('utf-8')).decode()  # base64 encoding
    href = f"data:text/csv;base64,{b64}"
    return f'<a href="{href}" download="{filename}">{text}</a>'

# Get dictionary of geolocation for id_coord
geo_map = pd.read_csv("metadata.csv")
geo_dict = {row['id_coord']: (row['lat'], row['lon']) for _, row in geo_map.iterrows()}

# Start the display on web app
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("CleanR - Detect Methane Leaks")
st.write("")
st.sidebar.header("Menu")
tabs = ["Prediction on uploaded images", "Leak detection map", "Leak track over time by id_coord", "Prediction detail"]
selected_tab = st.sidebar.radio("", tabs)


if selected_tab == "Prediction on uploaded images":
    file_up = st.file_uploader("Upload your images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)
    if file_up is not None:
        st.subheader("Prediction on uploaded images")
        all_predictions = pd.DataFrame() # create an empty DataFrame to store all predictions
        for img_file in file_up:
            image = Image.open(img_file)
            st.image(image, caption=img_file.name, use_column_width=True)
            st.write("")
            with st.spinner('Predicting...'):
                predictions_df = predict(img_file)
                # Append the current predictions to the DataFrame along with the id_coord, date and geolocation
                id_coord = img_file.name.split(".")[0][-7:]
                date = img_file.name.split("_")[0]
                latitude = geo_dict[id_coord][0]
                longitude = geo_dict[id_coord][1]
                predictions_df = [(id_coord,date,latitude,longitude,)+predictions_df[0]]
                all_predictions = all_predictions.append(predictions_df, ignore_index=True)
                # Print out the prediction labels with probability
                st.write("Prediction:", predictions_df[0][4], "-   Probability (%): ", round(predictions_df[0][5],0))
        if not all_predictions.empty:
            all_predictions.columns = ['Id_coord', "Date","Latitude","Longitude",'Label', 'Probability (%)']
            all_predictions.to_csv("predictions.csv", index=False)

if selected_tab == "Leak detection map":
    st.subheader("Leak detection map")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    # Create base map that will be centered around the mean latitude and longitude 
    m = folium.Map(location=[all_predictions.Latitude.mean(), all_predictions.Longitude.mean()], 
                 zoom_start=3, control_scale=True)
    # Loop through each row in the dataframe
    for i,row in all_predictions.iterrows():
        # Setup the content of the popup
        iframe = folium.IFrame('id:' + str(row["Id_coord"]))
        # Initialise the popup using the iframe
        popup = folium.Popup(iframe, min_width=300, max_width=300)
        # Add each row to the map
        folium.Marker(location=[row['Latitude'],row['Longitude']],
                  popup = popup, c=row['Id_coord'],
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(m)
    st_data = st_folium(m, width=700)

if selected_tab == "Leak track over time by id_coord":
    st.subheader("Leak track over time by id_coord")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    id_coord_list = list(all_predictions['Id_coord'].unique())
    selected_id_coord = st.sidebar.selectbox("Select an id_coord:", id_coord_list)
    filtered_predictions = all_predictions.loc[all_predictions['Id_coord'] == selected_id_coord]
    chart = alt.Chart(filtered_predictions).mark_line(color="#ff2b2b").encode(x='Date',y='Probability (%)')
    st.altair_chart(chart, use_container_width=True)

if selected_tab == "Prediction detail":
    st.subheader("Prediction Detail")
    all_predictions = pd.read_csv("predictions.csv")
    all_predictions['Date'] = pd.to_datetime(all_predictions['Date'], format='%Y%m%d')
    st.markdown(download_link(all_predictions, "prediction.csv", "Download CSV"), unsafe_allow_html=True)
    st.write("")
    st.write(all_predictions)

Overwriting app.py


In [50]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 3.251s
your url is: https://long-forks-do.loca.lt



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.229.190:8501

2023-05-14 11:51:30.322 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2023-05-14 11:51:36.073 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Argumen